<a href="https://colab.research.google.com/github/prashantiyaramareddy/AI-ML-Learnings/blob/master/ComputerVision/AlexNet_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### AlexNet Implementation in Pytorch

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
